In [93]:
import pandas as pd
import re, html
import hashlib
import json
from bs4 import BeautifulSoup
from datetime import datetime, timedelta
import time
from vaderSentiment.vaderSentiment import SentimentIntensityAnalyzer
from gdeltdoc import GdeltDoc, Filters
from sqlalchemy import create_engine, text
from sqlalchemy.types import CHAR, Float, Integer, JSON
import spacy

In [94]:
# ============================================================================
# CONFIGURATION
# ============================================================================
USER = "root"
PWD  = "2003"
HOST = "127.0.0.1"
PORT = 3306
DB   = "NewsVader"

ENGINE_URL = f"mysql+pymysql://{USER}:{PWD}@{HOST}:{PORT}/{DB}?charset=utf8mb4"

In [95]:
# ============================================================================
# CRÉATION DE LA BASE DE DONNÉES
# ============================================================================
def create_database():
    """Crée la base de données si elle n'existe pas"""
    ADMIN_URL = f"mysql+pymysql://{USER}:{PWD}@{HOST}:{PORT}/?charset=utf8mb4"
    admin_engine = create_engine(ADMIN_URL, future=True, pool_pre_ping=True)
    with admin_engine.begin() as conn:
        conn.exec_driver_sql(f"""
            CREATE DATABASE IF NOT EXISTS {DB}
            CHARACTER SET utf8mb4 COLLATE utf8mb4_general_ci;
        """)
    print(f" Base de données '{DB}' prête")

create_database()
engine = create_engine(ENGINE_URL, future=True, pool_pre_ping=True)

 Base de données 'NewsVader' prête


In [96]:
# ============================================================================
# SCHÉMAS DES TABLES
#  ============================================================================
# 
# TABLE 1: Articles détaillés avec NER et sentiment
DDL_ARTICLES = """
CREATE TABLE IF NOT EXISTS `articles` (
  `id` BIGINT UNSIGNED AUTO_INCREMENT PRIMARY KEY,
  `source` VARCHAR(255),
  `title` TEXT,
  `url` TEXT ,
  `url_hash` CHAR(32),
  `description` MEDIUMTEXT,
  `content` MEDIUMTEXT,
  `full_text` MEDIUMTEXT,
  `published_date` DATETIME NULL,
  `gdelt_date` DATETIME NULL,
  `language` VARCHAR(16),
  
  -- Sentiment
  `sentiment_compound` DOUBLE,
  `sentiment_pos` DOUBLE,
  `sentiment_neu` DOUBLE,
  `sentiment_neg` DOUBLE,
  `sentiment_label` VARCHAR(16),
  
  -- NER (stocké en JSON)
  `entities_persons` JSON,
  `entities_organizations` JSON,
  `entities_locations` JSON,
  `entities_other` JSON,
  `entities_count` INT DEFAULT 0,
  
  `created_at` TIMESTAMP DEFAULT CURRENT_TIMESTAMP,
  `updated_at` TIMESTAMP DEFAULT CURRENT_TIMESTAMP ON UPDATE CURRENT_TIMESTAMP,
  
  UNIQUE KEY `uk_url_hash` (`url_hash`),
  KEY `idx_published_date` (`published_date`),
  KEY `idx_gdelt_date` (`gdelt_date`),
  KEY `idx_sentiment_label` (`sentiment_label`)
) ENGINE=InnoDB DEFAULT CHARSET=utf8mb4 COLLATE=utf8mb4_general_ci;
"""

# TABLE 2: Agrégation quotidienne du sentiment
DDL_DAILY_SENTIMENT = """
CREATE TABLE IF NOT EXISTS `daily_sentiment` (
  `id` INT UNSIGNED AUTO_INCREMENT PRIMARY KEY,
  `date` DATE NOT NULL,
  `sentiment_mean` DOUBLE,
  `sentiment_median` DOUBLE,
  `sentiment_std` DOUBLE,
  `articles_count` INT,
  `positive_count` INT,
  `neutral_count` INT,
  `negative_count` INT,
  `top_entities` JSON,
  `updated_at` TIMESTAMP DEFAULT CURRENT_TIMESTAMP ON UPDATE CURRENT_TIMESTAMP,
  
  UNIQUE KEY `uk_date` (`date`),
  KEY `idx_date` (`date`)
) ENGINE=InnoDB DEFAULT CHARSET=utf8mb4 COLLATE=utf8mb4_general_ci;
"""

def create_tables():
    """Crée toutes les tables nécessaires"""
    with engine.begin() as conn:
        conn.exec_driver_sql(DDL_ARTICLES)
        print(" Table 'articles' créée/vérifiée")
        
        conn.exec_driver_sql(DDL_DAILY_SENTIMENT)
        print(" Table 'daily_sentiment' créée/vérifiée")

create_tables()

 Table 'articles' créée/vérifiée
 Table 'daily_sentiment' créée/vérifiée


In [97]:
# ============================================================================
# INITIALISATION DES ANALYSEURS
# ============================================================================
print("🔄 Chargement des modèles...")
analyzer = SentimentIntensityAnalyzer()
try:
    nlp = spacy.load("en_core_web_sm")
    print(" Modèle spaCy chargé")
except:
    print(" Installez spaCy: python -m spacy download en_core_web_sm")
    nlp = None

🔄 Chargement des modèles...
 Modèle spaCy chargé


In [98]:
# ============================================================================
# FONCTIONS UTILITAIRES
# ============================================================================

def clean_text_soft(text: str) -> str:
    """Nettoie le texte HTML et URLs"""
    if not isinstance(text, str):
        return ""
    text = html.unescape(text)
    text = BeautifulSoup(text, "html.parser").get_text(" ", strip=True)
    text = re.sub(r'(https?://\S+|www\.\S+)', ' ', text)
    text = re.sub(r'\s+', ' ', text).strip()
    return text

def label_from_compound(x: float) -> str:
    """Convertit score compound en label"""
    if pd.isna(x):
        return "Neutral"
    return "Positive" if x >= 0.05 else ("Negative" if x <= -0.05 else "Neutral")

def generate_url_hash(url: str) -> str:
    """Génère un hash MD5 de l'URL"""
    if not url:
        return ""
    return hashlib.md5(url.encode('utf-8')).hexdigest()


In [99]:
# ============================================================================
# ANALYSE NER (Named Entity Recognition)
# ============================================================================

def extract_entities(text: str) -> dict:
    """Extrait les entités nommées avec spaCy"""
    if not nlp or not text or not isinstance(text, str):
        return {
            "persons": [],
            "organizations": [],
            "locations": [],
            "other": [],
            "count": 0
        }
    
    try:
        doc = nlp(text[:100000])  # Limite à 100k caractères pour la performance
        
        entities = {
            "persons": [],
            "organizations": [],
            "locations": [],
            "other": []
        }
        
        for ent in doc.ents:
            entity_text = ent.text.strip()
            if len(entity_text) < 2:  # Ignore entités trop courtes
                continue
                
            if ent.label_ == "PERSON":
                entities["persons"].append(entity_text)
            elif ent.label_ in ["ORG", "PRODUCT"]:
                entities["organizations"].append(entity_text)
            elif ent.label_ in ["GPE", "LOC", "FAC"]:
                entities["locations"].append(entity_text)
            else:
                entities["other"].append(entity_text)
        
        # Déduplique et compte
        for key in entities:
            entities[key] = list(set(entities[key]))[:20]  # Max 20 par catégorie
        
        entities["count"] = sum(len(v) for v in entities.values())
        return entities
        
    except Exception as e:
        print(f" Erreur NER: {e}")
        return {"persons": [], "organizations": [], "locations": [], "other": [], "count": 0}


In [100]:

# ============================================================================
# RÉCUPÉRATION GDELT
# ============================================================================

def get_multiple_batches(num_batches=300):
    """Récupère les articles GDELT par lots"""
    gd = GdeltDoc()
    all_articles = []
    
    start_date = datetime(2017, 1, 1)
    end_date = datetime(2025, 9, 20)
    total_days = (end_date - start_date).days
    days_per_batch = total_days // num_batches
    current_date = start_date
    
    for i in range(num_batches):
        if i == num_batches - 1:
            period_end = end_date
        else:
            period_end = current_date + timedelta(days=days_per_batch)
            
        f = Filters(
            start_date=current_date.strftime("%Y-%m-%d"),
            end_date=period_end.strftime("%Y-%m-%d"),
            num_records=250,
            language="ENGLISH"
            #domain=["bloomberg.com", "theguardian.com", "ft.com", "economist.com"]
        )
        
        try:
            df_batch = gd.article_search(f)
            if not df_batch.empty:
                all_articles.append(df_batch)
                print(f" Batch {i+1}/{num_batches} ({current_date.strftime('%Y-%m-%d')} à {period_end.strftime('%Y-%m-%d')}): {len(df_batch)} articles")
        except Exception as e:
            print(f" Erreur batch {i+1}: {e}")
            
        current_date = period_end
        time.sleep(1)  # Rate limiting
    
    if all_articles:
        final_df = pd.concat(all_articles, ignore_index=True)
        final_df = final_df.drop_duplicates(subset=['url'], keep='first')
        print(f" Total: {len(final_df)} articles uniques")
        return final_df
    return pd.DataFrame()


In [101]:

# ============================================================================
# ANALYSE COMPLÈTE (SENTIMENT + NER)
# ============================================================================

def analyze_articles(df: pd.DataFrame) -> pd.DataFrame:
    """Applique sentiment + NER sur chaque article"""
    if df.empty:
        return df.copy()

    print(" Analyse des articles...")
    
    # Extraction des colonnes
    title = df.get("title", pd.Series([""]*len(df))).fillna("")
    content = df.get("content", pd.Series([""]*len(df))).fillna("")
    desc = df.get("description", pd.Series([""]*len(df))).fillna("")
    url = df.get("url", df.get("DocumentIdentifier", pd.Series([""]*len(df)))).fillna("")
    source = df.get("domain", df.get("sourceCommonName", pd.Series([""]*len(df)))).fillna("")
    lang = df.get("language", pd.Series([""]*len(df))).fillna("")
    
    # Dates
    published_raw = df.get("publishdate", df.get("date", pd.Series([None]*len(df))))
    gdelt_raw = df.get("seendate", pd.Series([None]*len(df)))
    
    # Parsing dates
    pub_dt = pd.to_datetime(published_raw, errors="coerce")
    seen_dt = pd.to_datetime(gdelt_raw, errors="coerce", format="%Y%m%dT%H%M%SZ")
    
    # Nettoyage texte
    full_text = (title + " " + content + " " + desc).map(clean_text_soft)
    
    # SENTIMENT
    print(" Analyse de sentiment...")
    scores = full_text.map(lambda t: analyzer.polarity_scores(t) if t else 
                           {"compound":0, "pos":0, "neu":1, "neg":0})
    
    # NER
    print(" Extraction des entités nommées...")
    entities = full_text.apply(extract_entities)
    
    # Construction du DataFrame final
    out = pd.DataFrame({
        "source": source.astype(str).str[:255],
        "url": url.astype(str),
        "url_hash": url.astype(str).map(generate_url_hash),
        "title": title.astype(str),
        "description": desc.astype(str),
        "content": content.astype(str),
        "full_text": full_text,
        "published_date": pub_dt,
        "gdelt_date": seen_dt,
        "language": lang.astype(str).str[:16],
        
        # Sentiment
        "sentiment_compound": scores.map(lambda s: s["compound"]),
        "sentiment_pos": scores.map(lambda s: s["pos"]),
        "sentiment_neu": scores.map(lambda s: s["neu"]),
        "sentiment_neg": scores.map(lambda s: s["neg"]),
        "sentiment_label": scores.map(lambda s: label_from_compound(s["compound"])),
        
        # Entités (JSON)
        "entities_persons": entities.map(lambda e: json.dumps(e["persons"])),
        "entities_organizations": entities.map(lambda e: json.dumps(e["organizations"])),
        "entities_locations": entities.map(lambda e: json.dumps(e["locations"])),
        "entities_other": entities.map(lambda e: json.dumps(e["other"])),
        "entities_count": entities.map(lambda e: e["count"])
    })
    
    print(f" Analyse terminée: {len(out)} articles traités")
    return out


In [102]:
# ============================================================================
# INSERTION EN BASE
# ============================================================================

def upsert_articles(df_scored: pd.DataFrame):
    """Insert/Update dans la table articles"""
    if df_scored.empty:
        print("⚠️ Aucun article à insérer")
        return 0

    # Conversion ligne par ligne pour gérer tous les types de NaN/NaT
    payload = []
    for _, row in df_scored.iterrows():
        clean_row = {}
        
        # Colonnes texte
        for col in ['source', 'url', 'url_hash', 'title', 'description', 'content', 'full_text', 'language',
                    'entities_persons', 'entities_organizations', 'entities_locations', 'entities_other', 'sentiment_label']:
            val = row.get(col)
            if pd.isna(val) or val is pd.NaT:
                clean_row[col] = None
            else:
                clean_row[col] = str(val) if val is not None else None
        
        # Colonnes dates - conversion explicite
        for col in ['published_date', 'gdelt_date']:
            val = row.get(col)
            if pd.isna(val) or val is pd.NaT:
                clean_row[col] = None
            else:
                try:
                    # Conversion en datetime Python natif
                    if hasattr(val, 'to_pydatetime'):
                        clean_row[col] = val.to_pydatetime()
                    else:
                        clean_row[col] = val
                except:
                    clean_row[col] = None
        
        # Colonnes numériques
        for col in ['sentiment_compound', 'sentiment_pos', 'sentiment_neu', 'sentiment_neg', 'entities_count']:
            val = row.get(col)
            if pd.isna(val):
                clean_row[col] = 0.0 if col != 'entities_count' else 0
            else:
                clean_row[col] = float(val) if col != 'entities_count' else int(val)
        
        payload.append(clean_row)
    
    try:
        with engine.begin() as conn:
            sql = text("""
                INSERT INTO articles
                  (source, url, url_hash, title, description, content, full_text,
                   published_date, gdelt_date, language,
                   sentiment_compound, sentiment_pos, sentiment_neu, sentiment_neg, sentiment_label,
                   entities_persons, entities_organizations, entities_locations, entities_other, entities_count)
                VALUES
                  (:source, :url, :url_hash, :title, :description, :content, :full_text,
                   :published_date, :gdelt_date, :language,
                   :sentiment_compound, :sentiment_pos, :sentiment_neu, :sentiment_neg, :sentiment_label,
                   :entities_persons, :entities_organizations, :entities_locations, :entities_other, :entities_count)
                ON DUPLICATE KEY UPDATE
                  title=VALUES(title),
                  description=VALUES(description),
                  content=VALUES(content),
                  full_text=VALUES(full_text),
                  published_date=VALUES(published_date),
                  gdelt_date=VALUES(gdelt_date),
                  sentiment_compound=VALUES(sentiment_compound),
                  sentiment_pos=VALUES(sentiment_pos),
                  sentiment_neu=VALUES(sentiment_neu),
                  sentiment_neg=VALUES(sentiment_neg),
                  sentiment_label=VALUES(sentiment_label),
                  entities_persons=VALUES(entities_persons),
                  entities_organizations=VALUES(entities_organizations),
                  entities_locations=VALUES(entities_locations),
                  entities_other=VALUES(entities_other),
                  entities_count=VALUES(entities_count)
            """)
            
            conn.execute(sql, payload)
        
        print(f"✅ {len(payload)} articles insérés/mis à jour dans 'articles'")
        return len(payload)
        
    except Exception as e:
        print(f"❌ Erreur insertion: {e}")
        raise

In [103]:
# ============================================================================
# AGRÉGATION QUOTIDIENNE
# ============================================================================

def compute_daily_sentiment():
    """Calcule et stocke les statistiques quotidiennes"""
    print("📊 Calcul des agrégations quotidiennes...")
    
    query = """
        SELECT 
            DATE(gdelt_date) as date,
            AVG(sentiment_compound) as sentiment_mean,
            STD(sentiment_compound) as sentiment_std,
            COUNT(*) as articles_count,
            SUM(CASE WHEN sentiment_label = 'Positive' THEN 1 ELSE 0 END) as positive_count,
            SUM(CASE WHEN sentiment_label = 'Neutral' THEN 1 ELSE 0 END) as neutral_count,
            SUM(CASE WHEN sentiment_label = 'Negative' THEN 1 ELSE 0 END) as negative_count,
            GROUP_CONCAT(DISTINCT entities_persons) as all_persons,
            GROUP_CONCAT(DISTINCT entities_organizations) as all_orgs,
            GROUP_CONCAT(DISTINCT entities_locations) as all_locs
        FROM articles
        WHERE gdelt_date IS NOT NULL
        GROUP BY DATE(gdelt_date)
        ORDER BY date
    """
    
    df_daily = pd.read_sql(query, engine)
    
    # Calcul de la médiane (pas disponible en SQL standard)
    median_query = """
        SELECT DATE(gdelt_date) as date, sentiment_compound
        FROM articles
        WHERE gdelt_date IS NOT NULL
        ORDER BY gdelt_date, sentiment_compound
    """
    df_for_median = pd.read_sql(median_query, engine)
    medians = df_for_median.groupby('date')['sentiment_compound'].median().reset_index()
    medians.columns = ['date', 'sentiment_median']
    
    df_daily = df_daily.merge(medians, on='date', how='left')
    
    # Top entités par jour (simplifié - à améliorer si besoin)
    df_daily['top_entities'] = df_daily.apply(
        lambda row: json.dumps({
            "persons": [],
            "organizations": [],
            "locations": []
        }), axis=1
    )
    
    # Insertion dans daily_sentiment
    try:
        with engine.begin() as conn:
            for _, row in df_daily.iterrows():
                sql = text("""
                    INSERT INTO daily_sentiment
                      (date, sentiment_mean, sentiment_median, sentiment_std, 
                       articles_count, positive_count, neutral_count, negative_count, top_entities)
                    VALUES
                      (:date, :sentiment_mean, :sentiment_median, :sentiment_std,
                       :articles_count, :positive_count, :neutral_count, :negative_count, :top_entities)
                    ON DUPLICATE KEY UPDATE
                      sentiment_mean=VALUES(sentiment_mean),
                      sentiment_median=VALUES(sentiment_median),
                      sentiment_std=VALUES(sentiment_std),
                      articles_count=VALUES(articles_count),
                      positive_count=VALUES(positive_count),
                      neutral_count=VALUES(neutral_count),
                      negative_count=VALUES(negative_count),
                      top_entities=VALUES(top_entities)
                """)
                conn.execute(sql, row.to_dict())
        
        print(f"✅ {len(df_daily)} jours agrégés dans 'daily_sentiment'")
        return df_daily
        
    except Exception as e:
        print(f"❌ Erreur agrégation: {e}")
        raise


In [ ]:
print("\n" + "="*70)
print("🚀 DÉMARRAGE DE L'ANALYSE GDELT")
print("="*70 + "\n")

# 1. Récupération des articles
print("📡 Récupération des articles GDELT...")
df = get_multiple_batches(num_batches=6)

if df.empty:
    print("❌ Aucun article récupéré")
else:
    # 2. Analyse (Sentiment + NER)
    scored = analyze_articles(df)

    # 3. Insertion dans la table articles
    upsert_articles(scored)

    # 4. Agrégation quotidienne
    daily = compute_daily_sentiment()

    # 5. Aperçu des résultats
    print("\n" + "="*70)
    print("📊 RÉSUMÉ DES RÉSULTATS")
    print("="*70)

    print(f"\n📰 Articles analysés: {len(scored)}")
    print(f"📅 Jours couverts: {len(daily)}")

    print("\n🎯 Distribution sentiment:")
    print(scored['sentiment_label'].value_counts())

    print("\n🏷️ Entités extraites (moyenne):")
    print(f"  - Personnes: {scored['entities_count'].mean():.1f} par article")

    print(f"\n✅ Données disponibles dans MariaDB:")
    print(f"   - Base: {DB}")
    print(f"   - Table 1: articles (détails + NER + sentiment)")
    print(f"   - Table 2: daily_sentiment (agrégation quotidienne)")
    print(f"   - URL: {ENGINE_URL}")

    print("\n" + "="*70)
    print(" TERMINÉ AVEC SUCCÈS!")
    print("="*70 + "\n")


🚀 DÉMARRAGE DE L'ANALYSE GDELT

📡 Récupération des articles GDELT...
 Batch 1/6 (2017-01-01 à 2018-06-15): 250 articles
 Batch 2/6 (2018-06-15 à 2019-11-27): 250 articles
 Batch 3/6 (2019-11-27 à 2021-05-10): 250 articles
 Batch 4/6 (2021-05-10 à 2022-10-22): 250 articles
 Batch 5/6 (2022-10-22 à 2024-04-04): 250 articles
 Batch 6/6 (2024-04-04 à 2025-09-20): 250 articles
 Total: 1499 articles uniques
 Analyse des articles...
 Analyse de sentiment...
 Extraction des entités nommées...
 Analyse terminée: 1500 articles traités
✅ 1500 articles insérés/mis à jour dans 'articles'
📊 Calcul des agrégations quotidiennes...
✅ 178 jours agrégés dans 'daily_sentiment'

📊 RÉSUMÉ DES RÉSULTATS

📰 Articles analysés: 1500
📅 Jours couverts: 178

🎯 Distribution sentiment:
sentiment_label
Neutral     954
Positive    284
Negative    262
Name: count, dtype: int64

🏷️ Entités extraites (moyenne):
  - Personnes: 1.2 par article

✅ Données disponibles dans MariaDB:
   - Base: NewsVader
   - Table 1: article